In [1]:
import import_ipynb
import sys
sys.path.append('C:\\Users\\USER\\JupyterProjects\\DTML')
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from Stock_Dataset import StockDataset
import argparse
from AttLSTM import att_LSTM
from Transformer_Encoder import Transformer
import numpy as np
import time
from metric import metric_acc as ACC
from metric import metric_mcc as MCC
import matplotlib.pyplot as plt
import csv
import os
from loss_fn1 import Selective_Regularization

def train(att_LSTM,transformer,                                  ## Model
          att_LSTM_optimizer, transformer_optimizer,   ## Optimizer
          Partition, args):                                      ## Data, loss function, argument
    trainloader = DataLoader(Partition['train'],
                             batch_size = args.batch_size,
                             shuffle=False, drop_last=True)

    att_LSTM.train()
    transformer.train()

    train_loss = 0.0
    for i, (x,y) in enumerate(trainloader):
        data_out_list = []
        for i in range(len(x)):

            att_LSTM_optimizer.zero_grad()
            transformer_optimizer.zero_grad()
            
            x_input = x[i].to(args.device)
            
            if i == 1:
                true_y = y[i].squeeze().float().to(args.device)

            att_LSTM.hidden = [hidden.to(args.device) for hidden in att_LSTM.init_hidden()]
            
            # 'list' object has no attribute 'float'

            hidden_context, attn_weights = att_LSTM(x_input)

            data_out_list.append(hidden_context)

        index_output = data_out_list[0]  # torch.Size([128, 10])
        stock_output = data_out_list[1] # torch.Size([128, 10])
        

        Norm_ = nn.LayerNorm(10, device=args.device)
        
        index_output = args.r * Norm_(index_output) + args.b
        stock_output = args.r * Norm_(stock_output) + args.b


        Transformer_input = index_output* args.market_beta + stock_output
        

        output1,wp,bp = transformer(Transformer_input.transpose(0,1).unsqueeze(2))


        # output_ = torch.where(output1 >= 0.5, 1.0, 0.0)
        # output_.requires_grad=True
        
        loss = args.loss_fn(output1, true_y, wp.squeeze(), bp)
        loss.backward()

        att_LSTM_optimizer.step() ## parameter 갱신
        transformer_optimizer.step()

        train_loss += loss.item()

    train_loss = train_loss / len(trainloader)
    return att_LSTM, transformer, train_loss


def validation(att_LSTM,transformer,
               partition, args):
    valloader = DataLoader(partition['val'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    att_LSTM.eval()
    transformer.eval()
    val_loss = 0.0

    with torch.no_grad():
        for i, (x, y) in enumerate(valloader):

            data_out_list = []
            for i in range(len(x)):
                
                x_input = x[i].to(args.device)

                if i==1:
                    true_y = y[i].squeeze().float().to(args.device)
                att_LSTM.hidden = [hidden.to(args.device) for hidden in att_LSTM.init_hidden()]


                hidden_context, attn_weights = att_LSTM(x_input)

                data_out_list.append(hidden_context)

            index_output = data_out_list[0]  # torch.Size([128, 10])
            stock_output = data_out_list[1]  # torch.Size([128, 10])

            Norm_ = nn.LayerNorm(10, device=args.device)
            index_output = args.r * Norm_(index_output) + args.b
            stock_output = args.r * Norm_(stock_output) + args.b

            Transformer_input = index_output* args.market_beta  + stock_output

            output1,wp,bp = transformer(Transformer_input.transpose(0,1).unsqueeze(2))

            loss = args.loss_fn(output1, true_y, wp.squeeze(), bp)
            val_loss += loss.item()

        val_loss = val_loss / len(valloader)
        return att_LSTM, transformer, val_loss


def test(att_LSTM, transformer,
               partition, args):
    testloader = DataLoader(partition['test'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    att_LSTM.eval()
    transformer.eval()

    ACC_metric = 0.0
    MCC_metric = 0.0
    with torch.no_grad():
        for i, (x, y) in enumerate(testloader):

            data_out_list = []
            for i in range(len(x)):
                
                x_input = x[i].to(args.device)
                # feature transform
                if i==1:
                    true_y = y[i].squeeze().float().to(args.device)

                att_LSTM.hidden = [hidden.to(args.device) for hidden in att_LSTM.init_hidden()]


                hidden_context, attn_weights = att_LSTM(x_input)

                data_out_list.append(hidden_context)

            index_output = data_out_list[0]  # torch.Size([128, 10])
            stock_output = data_out_list[1]  # torch.Size([128, 10])

            Norm_ = nn.LayerNorm(10, device=args.device)
            index_output = args.r * Norm_(index_output) + args.b
            stock_output = args.r * Norm_(stock_output) + args.b

            Transformer_input = index_output * args.market_beta + stock_output

            output1,wp,bp = transformer(Transformer_input.transpose(0,1).unsqueeze(2))

            output_ = torch.where(output1 >= 0.5, 1.0, 0.0)

            output_.requires_grad = True

            ACC_metric += ACC(output_, true_y)
            MCC_metric += MCC(output_, true_y)

        ACC_metric = ACC_metric / len(testloader)
        MCC_metric = MCC_metric / len(testloader)

        return ACC_metric, MCC_metric



def experiment(partition, args):
    att_LSTM = args.att_LSTM(args.input_dim, args.hid_dim, args.output_dim, args.num_layers, args.batch_size,
                             args.dropout, args.use_bn, args.attention_head, args.attn_size, activation="ReLU")
    transformer = args.transformer(args.trans_feature_size, args.trans_num_laysers,args.dropout, args.batch_size, args.x_frames, args.trans_nhead)
    
    
    args.r = nn.init.xavier_normal_(torch.empty(args.batch_size, args.hid_dim)).to(args.device) 
    args.b = torch.zeros(1).to(args.device)
    
    att_LSTM.to(args.device)
    transformer.to(args.device)

    if args.optim == 'SGD':
        att_LSTM_optimizer = optim.SGD(att_LSTM.parameters(), lr=args.lr, weight_decay=args.l2)
        transformer_optimizer = optim.SGD(transformer.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        att_LSTM_optimizer = optim.RMSprop(att_LSTM.parameters(), lr=args.lr, weight_decay=args.l2)
        transformer_optimizer = optim.RMSprop(transformer.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        att_LSTM_optimizer = optim.Adam(att_LSTM.parameters(), lr=args.lr, weight_decay=args.l2)
        transformer_optimizer = optim.Adam(transformer.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')

    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    # ===================================== #
    for epoch in range(args.epoch):
        ts = time.time()
        att_LSTM, transformer, train_loss = train(att_LSTM, transformer,
                                                  att_LSTM_optimizer, transformer_optimizer,
                                                  partition, args)

        att_LSTM, transformer, val_loss = validation(att_LSTM, transformer, partition, args)

        te = time.time()

        ## 각 에폭마다 모델을 저장하기 위한 코드
        torch.save(att_LSTM.state_dict(), args.new_file_path + '\\' + str(epoch) +'_epoch' +'_att_LSTM' +'.pt')
        torch.save(transformer.state_dict(), args.new_file_path + '\\' + str(epoch) +'_epoch' +'_transformer' +'.pt')

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        print('Epoch {}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
              .format(epoch, train_loss, val_loss, te - ts))

    ## val_losses에서 가장 값이 최소인 위치를 저장함
    site_val_losses = val_losses.index(min(val_losses)) ## 10 epoch일 경우 0번째~9번째 까지로 나옴
    att_LSTM = args.att_LSTM(args.input_dim, args.hid_dim, args.output_dim, args.num_layers, args.batch_size,
                             args.dropout, args.use_bn, args.attention_head, args.attn_size,
                             activation="ReLU")
    transformer = args.transformer(args.trans_feature_size, args.trans_num_laysers,
                                   args.dropout, args.batch_size, args.x_frames, args.trans_nhead)
    att_LSTM.to(args.device)
    transformer.to(args.device)

    att_LSTM.load_state_dict(torch.load(args.new_file_path + '\\' + str(site_val_losses) +'_epoch' +'_att_LSTM'+ '.pt'))
    transformer.load_state_dict(torch.load(args.new_file_path + '\\' + str(site_val_losses) + '_epoch' + '_transformer' + '.pt'))

    ACC,MCC = test(att_LSTM, transformer, partition, args)
    print('ACC: {}, MCC: {}'.format(ACC, MCC))

    with open(args.new_file_path + '\\'+ str(site_val_losses)+'Epoch_test_metric' +'.csv', 'w') as fd:
        print('ACC: {}, MCC: {}'.format(ACC, MCC), file=fd)

    result = {}

    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['ACC'] = ACC
    result['MCC'] = MCC

    return vars(args), result


importing Jupyter notebook from Stock_Dataset.ipynb
importing Jupyter notebook from feature_extract.ipynb
importing Jupyter notebook from AttLSTM.ipynb
importing Jupyter notebook from attention.ipynb
importing Jupyter notebook from Transformer_Encoder.ipynb
importing Jupyter notebook from metric.ipynb
importing Jupyter notebook from loss_fn1.ipynb


In [2]:
# '^KS11' : KOSPI
# '^KQ11' : 코스닥
# '^IXIC' : 나스닥
# '^GSPC' : SNP 500 지수
# '^DJI' : 다우존수 산업지수
# '^HSI' : 홍콩 항생 지수
# '^N225' : 니케이지수
# '^GDAXI' : 독일 DAX
# '^FTSE' : 영국 FTSE
# '^FCHI' : 프랑스 CAC
# '^IBEX' : 스페인 IBEX
# '^TWII' : 대만 기권
# '^AEX' : 네덜란드 AEX
# '^BSESN' : 인도 센섹스
# 'RTSI.ME' : 러시아 RTXI
# '^BVSP' : 브라질 보베스파 지수
# 'GC=F' : 금 가격
# 'CL=F' : 원유 가격 (2000/ 8 / 20일 부터 데이터가 있음)
# 'BTC-USD' : 비트코인 암호화폐
# 'ETH-USD' : 이더리움 암호화폐

# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

# ========= experiment setting ========== #
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.save_file_path = "C:\\Users\\USER\\JupyterProjects\\DTML\\result"

## ======== data ============= #
args.index = ['^IXIC']
# [] 되는거
# ['CHL', 'DCM' , 'DOW', 'NTT' , 'SYT', 'TOT'] 안되는거
args.stock_list =  ['AAPL', 'AMZN', 'BA', 'BAC', 'BHP', 'BRK-B', 'CMCSA', 'CVX', 'D','DIS','DUK', 'EXC', 'GE', 'GOOGL', 'HD', 'INTC', 'JNJ', 'JPM', 'KO', 'MA', 'MMM', 'MO', 'MRK', 'MSFT', 'NGG','NVS', 'ORCL', 'PEP', 'PFE', 'PG', 'PTR', 'RDS-B', 'RIO', 'SO', 'SPY', 'T', 'TM', 'UNH', 'UPS', 'VALE', 'VZ', 'WFC', 'WMT', 'XOM']
#args.stock_list =  ['AMZN']
#, 'BA', 'BAC']
args.data_count = len(args.stock_list)
args.train_start = "2007-01-03"
args.train_end = "2015-01-01"
args.validation_start = "2015-01-01"
args.validation_end = "2016-01-03"
args.test_start = "2016-01-04"
args.test_end = "2016-12-31"

# ====== hyperparameter ======= #
args.batch_size = 128
args.x_frames = 10
args.y_frames = 1
args.input_dim = 10
args.output_dim = 1
args.dropout = 0.15
args.use_bn = True
args.loss_fn = Selective_Regularization  ## loss function for classification : cross entropy
args.optim = 'Adam'
args.lr = 0.0005
args.l2 = 0.00001 #?
args.epoch = 100
# ============= model ================== #
args.att_LSTM = att_LSTM
args.transformer = Transformer

# ====== att_lstm hyperparameter ======= #
args.hid_dim = 10
args.attention_head = 1
args.attn_size = 10
args.num_layers = 1
args.decoder_x_frames = 1

# ====== transformer hyperparameter ======= #
args.trans_feature_size = 250
args.trans_num_laysers = 1
args.trans_nhead = 10
args.market_beta = 0.1



In [3]:
with open(args.save_file_path + '\\' + 'DTML_result_t.csv', 'w', encoding='utf-8', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(["model", "stock", "entire_exp_time",  "avg_test_ACC", "avg_test_MCC"])

    for stock in args.stock_list:
        est = time.time()
        setattr(args, 'symbol', stock)
        args.new_file_path = args.save_file_path + '\\' + "DTML_" + args.symbol
        os.makedirs(args.new_file_path)

        args.entire_datalist = args.index + [stock]
        # 0번째에 index 1번째에 stock 1개가 input으로 들어감
        trainset = StockDataset(args.entire_datalist, args.x_frames, args.y_frames,
                                args.train_start, args.train_end)
        valset = StockDataset(args.entire_datalist, args.x_frames, args.y_frames,
                              args.validation_start,args.validation_end)
        testset = StockDataset(args.entire_datalist, args.x_frames, args.y_frames,
                               args.test_start, args.test_end)
        partition = {'train': trainset, 'val': valset, 'test': testset}


        setting, result = experiment(partition, args)
        eet = time.time()
        entire_exp_time = eet - est

        fig = plt.figure()
        plt.plot(result['train_losses'])
        plt.plot(result['val_losses'])
        plt.legend(['train_losses', 'val_losses'], fontsize=15)
        plt.xlabel('epoch', fontsize=15)
        plt.ylabel('loss', fontsize=15)
        plt.grid()
        plt.savefig(args.new_file_path + '\\' + str(args.symbol) + '_fig' + '.png')
        plt.close(fig)

        # csv파일에 기록하기
        wr.writerow(["DTML", args.symbol,entire_exp_time, result['ACC'], result['MCC']])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Epoch 0, Loss(train/val) 5.06375/5.05429. Took 1.21 sec
Epoch 1, Loss(train/val) 5.05816/5.04938. Took 1.01 sec
Epoch 2, Loss(train/val) 5.05228/5.04945. Took 1.00 sec
Epoch 3, Loss(train/val) 5.05286/5.04753. Took 1.01 sec
Epoch 4, Loss(train/val) 5.05236/5.04721. Took 1.01 sec
Epoch 5, Loss(train/val) 5.04989/5.04637. Took 1.00 sec
Epoch 6, Loss(train/val) 5.05024/5.04536. Took 1.00 sec
Epoch 7, Loss(train/val) 5.05090/5.04238. Took 1.02 sec
Epoch 8, Loss(train/val) 5.04875/5.04046. Took 1.01 sec
Epoch 9, Loss(train/val) 5.04685/5.03928. Took 1.02 sec
Epoch 10, Loss(train/val) 